In [15]:
from random import seed, choice

import os
import heapq
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from skbio.tree import TreeNode

### Overview
**Goal**: Generate a smaller group of clades from the original 10,575 genomes in the phylogeny, whereby the divergence amongst the clades can be limited to a given threshold, and the divergence between the clades is maximized.

This approach is motivated by [WoL](https://github.com/biocore/wol/blob/master/code/notebooks/taxon_subsampling.ipynb) (Zhu et al.)

First, we limit the possible groupings of clades to be chosen from those that have a minimum relative evolutionary divergence (**RED**) ([Parks, et al., 2018](https://www.nature.com/articles/nbt.4229)) over a given threshold, `min_red`.
Then, for each candidate grouping rooted at `clade_root`, to maximize the divergence between clades, we can find the `n_clades` by finding the `n` descendants of `clade_root` that minimize $\sum_{i=1}^n \text{RED}(\texttt{node}_i)$. Specific genomes can then be sampled based on criteria such as:
1. Contains the most marker genes.
2. Contamination level is the lowest.
3. DNA quality score is the highest.
4. Random selection.
5. Sampling of all included genomes.

Qiyun has previously used the first three in unison to select a single. Those three criteria could potentially be used to filter to a smaller list, amongst which all genomes are sampled.

**Effect of paramters**:
* Increasing `min_red` will limit the number of genomes a given group of clades can contain.
* Increasing `n_clades` will increase the number of subgroups of genomes that are created.
Increasing either paramter will increase the resolution that downstream methods need to properly handle such groups of genomes.

In [16]:
seed(42)
%matplotlib inline

In [17]:
tree_fp = 'https://raw.githubusercontent.com/biocore/wol/master/data/trees/astral/branch_length/cons/astral.cons.nid.nwk'
# tree_fp = 'data/trees/astral.nid.nwk'
tree = TreeNode.read(tree_fp)
tree.count(tips=True)

10575

In [18]:
supports_fp = 'https://raw.githubusercontent.com/biocore/wol/master/data/trees/astral/astral.supports.tsv.bz2'
dfs = pd.read_table(supports_fp, index_col=0)

In [19]:
dfs.head()

,EN,LPP,QT
#node,,,
N2,196.0,0.998406,0.450953
N3,196.0,0.998406,0.450953
N4,124.0,0.999993,0.535120
N5,195.0,0.914387,0.398648
N6,208.0,1.000000,0.539983


In [20]:
genomes_fp = 'https://biocore.github.io/wol/data/genomes/metadata.tsv.bz2'
dfg = pd.read_table(genomes_fp, index_col=0)

In [21]:
dfg.head()

,asm_name,assembly_accession,bioproject,biosample,wgs_master,seq_rel_date,submitter,ftp_path,img_id,gtdb_id,...,coding_density,completeness,contamination,strain_heterogeneity,markers,5s_rrna,16s_rrna,23s_rrna,trnas,draft_quality
#genome,,,,,,,,,,,,,,,,,,,,,
G000005825,ASM582v2,GCF_000005825.2,PRJNA224116,SAMN02603086,NaN,2010/12/15,"Center for Genomic Sciences, Allegheny-Singer ...",ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000...,646311908,RS_GCF_000005825.2,...,85.144124,98.68,1.32,0.0,377,yes,yes,yes,20,high
G000006175,ASM617v2,GCF_000006175.1,PRJNA224116,SAMN00000040,NaN,2010/06/03,US DOE Joint Genome Institute (JGI-PGF),ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000...,646564549,RS_GCF_000006175.1,...,80.167033,99.05,0.00,0.0,165,no,yes,yes,19,medium
G000006605,ASM660v1,GCF_000006605.1,PRJNA224116,SAMEA3283089,NaN,2005/06/27,Bielefeld Univ,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000...,637000085,RS_GCF_000006605.1,...,89.378688,100.00,0.68,0.0,319,yes,yes,yes,20,high
G000006725,ASM672v1,GCF_000006725.1,PRJNA224116,SAMN02603773,NaN,2004/06/04,Sao Paulo state (Brazil) Consortium,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000...,637000348,RS_GCF_000006725.1,...,82.592990,99.59,0.18,0.0,325,yes,yes,yes,20,high
G000006745,ASM674v1,GCF_000006745.1,PRJNA57623,SAMN02603969,NaN,2001/01/09,TIGR,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000...,637000333,RS_GCF_000006745.1,...,86.533164,99.86,0.03,0.0,360,yes,yes,yes,20,high


In [22]:
dfg.insert(0, '# assembly_accession', dfg['assembly_accession'])

In [23]:
dfg.head()

,# assembly_accession,asm_name,assembly_accession,bioproject,biosample,wgs_master,seq_rel_date,submitter,ftp_path,img_id,...,coding_density,completeness,contamination,strain_heterogeneity,markers,5s_rrna,16s_rrna,23s_rrna,trnas,draft_quality
#genome,,,,,,,,,,,,,,,,,,,,,
G000005825,GCF_000005825.2,ASM582v2,GCF_000005825.2,PRJNA224116,SAMN02603086,NaN,2010/12/15,"Center for Genomic Sciences, Allegheny-Singer ...",ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000...,646311908,...,85.144124,98.68,1.32,0.0,377,yes,yes,yes,20,high
G000006175,GCF_000006175.1,ASM617v2,GCF_000006175.1,PRJNA224116,SAMN00000040,NaN,2010/06/03,US DOE Joint Genome Institute (JGI-PGF),ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000...,646564549,...,80.167033,99.05,0.00,0.0,165,no,yes,yes,19,medium
G000006605,GCF_000006605.1,ASM660v1,GCF_000006605.1,PRJNA224116,SAMEA3283089,NaN,2005/06/27,Bielefeld Univ,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000...,637000085,...,89.378688,100.00,0.68,0.0,319,yes,yes,yes,20,high
G000006725,GCF_000006725.1,ASM672v1,GCF_000006725.1,PRJNA224116,SAMN02603773,NaN,2004/06/04,Sao Paulo state (Brazil) Consortium,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000...,637000348,...,82.592990,99.59,0.18,0.0,325,yes,yes,yes,20,high
G000006745,GCF_000006745.1,ASM674v1,GCF_000006745.1,PRJNA57623,SAMN02603969,NaN,2001/01/09,TIGR,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000...,637000333,...,86.533164,99.86,0.03,0.0,360,yes,yes,yes,20,high


In [24]:
dfg_c = dfg.copy()
dfg_c = dfg_c.set_index('# assembly_accession')

In [25]:
dfg_c.to_csv('data/wol_supplemental_metadata.tsv', sep='\t')

In [26]:
tree_tips = [node.name for node in tree.tips()]

In [27]:
dfg['refseq_category'].value_counts()

na                       5567
representative genome    4904
reference genome          104
Name: refseq_category, dtype: int64

In [28]:
rep_ref_genomes = dfg.loc[dfg['refseq_category'].isin(['representative genome', 'reference genome'])].index

In [29]:
def calc_brlen_metrics(tree):
    """Calculate branch length-related metrics.
    
    Originally from Zhu et al.

    Parameters
    ----------
    tree : skbio.TreeNode

    Notes
    -----
    The following metrics are calculated:
    
        - height: Sum of branch lengths from the root to the node.

        - depths: Sums of branch lengths from all descendants to current node.

        - red: Relative evolutionary divergence (RED), introduced by Parks,
          et al., 2018, Nat Biotechnol.

              RED = p + (d / u) * (1 - p)

          where p = RED of parent, d = length, u = mean depth of parent
    
    Metrics will be appended to each node of tree in place.
    """
    # calculate depths
    for node in tree.postorder(include_self=True):
        if node.name is None:
            raise ValueError('Error: Found an unnamed node.')
        if node.length is None:
            node.length = 0.0
        if node.is_tip():
            node.depths = [0.0]
            node.taxa = [node.name]
        else:
            node.depths = [
                y + x.length for x in node.children for y in x.depths]
            node.taxa = sorted(set().union(*[x.taxa for x in node.children]))

    # calculate heights and REDs
    for node in tree.preorder(include_self=True):
        if node.is_root():
            node.height = 0.0
            node.red = 0.0
        else:
            node.height = node.parent.height + node.length
            if node.is_tip():
                node.red = 1.0
            else:
                node.red = node.parent.red + node.length \
                    / (node.length + sum(node.depths) / len(node.depths)) \
                    * (1 - node.parent.red)

In [30]:
calc_brlen_metrics(tree)

In [259]:
# value_counts = [dfg.loc[node.taxa]['lv2_group'].value_counts()
#                 for node in groupings[0]]

In [260]:
len(groupings)

1917

## Sample genomes from within clade by WoL criteria:


Criterium 1: Most marker genes.

In [261]:
g2markers = dfg['markers'].to_dict()

In [262]:
def most_markers(gs, g2markers):
    max_gs = []
    max_markers = 0
    for g in sorted(gs, key=lambda x: g2markers[x], reverse=True):
        if max_markers == 0:
            max_markers = g2markers[g]
            max_gs.append(g)
        elif max_markers == g2markers[g]:
            max_gs.append(g)
        else:
            break
        max_gs.append(g)
    return max_gs, max_markers

Criterium 2: Lowest contamination.

In [263]:
g2contam = dfg['contamination'].to_dict()

In [264]:
def least_contaminated(gs, g2contam):
    min_gs = []
    min_contam = None
    for g in sorted(gs, key=lambda x: g2contam[x]):
        if min_contam is None:
            min_contam = g2contam[g]
            min_gs.append(g)
        elif min_contam == g2contam[g]:
            min_gs.append(g)
        else:
            break
    return min_gs, min_contam

Criterium 3: Highest DNA quality score.

In [265]:
g2dnaqty = dfg['score_fna'].to_dict()

In [266]:
def best_dna(gs, g2dnaqty):
    max_gs = []
    max_dnaqty = 0
    for g in sorted(gs, key=lambda x: g2dnaqty[x], reverse=True):
        if max_dnaqty == 0:
            max_dnaqty = g2dnaqty[g]
            max_gs.append(g)
        elif max_dnaqty == g2dnaqty[g]:
            max_gs.append(g)
        else:
            break
    return max_gs, max_dnaqty

In [267]:
g2species = dfg['species'].to_dict()

In [268]:
g2refseq_category = dfg['refseq_category'].to_dict()

In [269]:
dfg['refseq_category'].value_counts()

na                       5567
representative genome    4904
reference genome          104
Name: refseq_category, dtype: int64

In [270]:
categories = ['representative genome', 'reference genome']
def is_complete_genome(gs, g2refseq_category):
    new_gs = g2refseq_category.loc[gs]
    new_gs = new_gs.loc[new_gs['refseq_category'].isin(categories)]
    new_gs = list(new_gs.index)
    if len(new_gs) > 0:
        return new_gs, True
    else:
        return gs, False
        

In [284]:
from itertools import product

In [295]:
min_reds = [0.5, 0.75, 0.9] # between 0 and 1
n_clades_s = [5, 10, 15, 20]
save = True
reads_per_class = 10 ** 4
output_dir = 'data/subsets'

## start loop
for min_red, n_clades in product(min_reds, n_clades_s):
    clade_roots = []
    candidates = [tree]
    while candidates:
        node = candidates.pop()
        if node.red >= min_red:
            clade_roots.append(node)
        else:
            candidates.extend(child for child in node)

    tree_dict = {node.name: node for node in tree.traverse()}

    groupings = []
    for clade_root in clade_roots:

        if len(list(clade_root.tips())) < n_clades:
            groupings.append([])
            continue

        chosen_clades = [(sum(node.red for node in clade_root),
                         clade_root.name)]

        while len(chosen_clades) < n_clades:
            # if chosen_clades[0][0] < 3:
            red_score, node_name = heapq.heappop(chosen_clades)
            node = tree_dict[node_name] 
            if node.children:
                for child in node:
                    heapq.heappush(chosen_clades,
                                   (sum(gc.red for gc in child),
                                    child.name))
            else:
                heapq.heappush(chosen_clades, (3, node.name))

        groupings.append([tree_dict[node_name] for 
                          _, node_name in chosen_clades])


    print("min-red: {}\tn_clades: {}\tn_groups: {}".format(min_red, n_clades, len(groupings)))

    genomes_groups = []
    for i, group in enumerate(groupings):
        # print("\nGroup {}: n_descendants {}".format(i, sum(len(list(node.tips())) for node in group)))
        chosen_clades = [node.name for node in group]
        chosen_genomes = dict()
        for nid in chosen_clades:
            gs = tree_dict[nid].taxa

            gs, complete = is_complete_genome(gs, dfg)
            if not complete:
                # print("Warning: {} has no complete child genomes".format(nid))
                continue
            # maximize marker count

            gs, max_markers = most_markers(gs, g2markers)

            if len(gs) > 1:
                # minimize contamination
                gs, min_contam = least_contaminated(gs, g2contam)

            if len(gs) > 1:
                # maximize DNA quality
                gs, max_dnaqty = best_dna(gs, g2dnaqty)

            if len(gs) > 1:
                # random choice (UPDATE: choose first one for stability)
                # print('Clade %s: Equally good: %s.' % (nid, ', '.join(gs)))
                g = gs[0] # choice(gs)

            else:
                g = max(gs)

            chosen_genomes[nid] = g

        if len(chosen_genomes) < n_clades:
            continue
        # print("\nGroup {}: n_descendants {}".format(i, sum(len(list(node.tips())) for node in group)))

        genomes_groups.append(chosen_genomes.items())
        add_rows = []
        for nid, g in sorted(chosen_genomes.items(), key=lambda x: g2species[x[1]]):
            add_rows.append([nid, g, reads_per_class, g2species[g]])
            # print('%s - %s: %s' % (nid, g, g2species[g]))

        # get a version of tree sheared to 
        #  a) the tree only containg selected genomes
        if save:
            try:
                sheared_minimal = tree.shear(list(chosen_genomes.values()))
                tree_name = 'min-red_{}__n_{}__group_{}__minimal-tree.nwk'.format(min_red, n_clades, i)
                # # can run the following if you want tips to 
                # invert_nodes = {val: key for key, val in chosen_genomes.items()}
                # for tip in sheared_minimal.tips():
                #     tip.name = invert_nodes[tip.name]
                out_filename = os.path.join(output_dir, tree_name)
                sheared_minimal.write(out_filename)
                #  b) the tree that has all nodes descending from selected clades
                all_descendants = [tree_dict[node].tips() for node in chosen_genomes.keys()]
                sheared_all_descendants = tree.shear([item.name for sublist in all_descendants for item in sublist])
                tree_name = 'min-red_{}__n_{}__group_{}__all-desc-tree.nwk'.format(min_red, n_clades, i)
                out_filename = os.path.join(output_dir, tree_name)
                sheared_all_descendants.write(out_filename)


                df = pd.DataFrame(add_rows, columns=['class', '#genome', 'n_reads', 'species_name'])
                filtered_dfg = dfg.loc[dfg.index.isin(df['#genome'])].to_dict()
                accessions = [filtered_dfg['assembly_accession'][genome] for genome in df['#genome']]
                # output_dir = 'data'
                df['id'] = accessions
                df.drop(['#genome'], axis=1, inplace=True)
                f_name = 'min-red_{}__n_{}__group_{}__genome-ids.txt'.format(min_red, n_clades, i)
                out_filename = os.path.join(output_dir, f_name)
                df.to_csv(out_filename, index=None, sep='\t')
            except:
                pass

min-red: 0.5	n_clades: 5	n_groups: 443
min-red: 0.5	n_clades: 10	n_groups: 443
min-red: 0.5	n_clades: 15	n_groups: 443
min-red: 0.5	n_clades: 20	n_groups: 443
min-red: 0.75	n_clades: 5	n_groups: 1917
min-red: 0.75	n_clades: 10	n_groups: 1917
min-red: 0.75	n_clades: 15	n_groups: 1917
min-red: 0.75	n_clades: 20	n_groups: 1917
min-red: 0.9	n_clades: 5	n_groups: 4700
min-red: 0.9	n_clades: 10	n_groups: 4700
min-red: 0.9	n_clades: 15	n_groups: 4700
min-red: 0.9	n_clades: 20	n_groups: 4700


In [310]:
for min_red, n_clades in product(min_reds, n_clades_s):
    print("min-red: {}\tn_clades: {}".format(min_red, n_clades))
    in_tree = "data/subsets/min-red_{}__n_{}__group_0__all-desc-tree.nwk".format(min_red, n_clades)
    out_dir = "taxdumps/min-red_{}__n_{}__group_0_taxdump".format(min_red, n_clades)
    %run -i ./tree_to_taxonomy.py $in_tree --out-directory $out_dir


min-red: 0.5	n_clades: 5
Number of nodes: 1605.
min-red: 0.5	n_clades: 10
Number of nodes: 1603.
min-red: 0.5	n_clades: 15
Number of nodes: 1603.
min-red: 0.5	n_clades: 20
Number of nodes: 1603.
min-red: 0.75	n_clades: 5
Number of nodes: 389.
min-red: 0.75	n_clades: 10
Number of nodes: 387.
min-red: 0.75	n_clades: 15
Number of nodes: 387.
min-red: 0.75	n_clades: 20
Number of nodes: 387.
min-red: 0.9	n_clades: 5
Number of nodes: 73.
min-red: 0.9	n_clades: 10
Number of nodes: 73.
min-red: 0.9	n_clades: 15
Number of nodes: 69.
min-red: 0.9	n_clades: 20


FileNotFoundError: [Errno 2] No such file or directory: 'data/subsets/min-red_0.9__n_20__group_0__all-desc-tree.nwk'

In [299]:
%run -i ./tree_to_taxonomy.py  --out-directory 

Number of nodes: 1605.
